# Imports

In [128]:
import pandas as pd

Needed to load data saved in data folder

In [129]:
import os
import sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

from src.utils.utils import is_datetime


# Utils

In [130]:
def split_mandante_visitante(row):
    hor_res = row['Hor/Res']
    error = False
    try:
        pontos_mandante = int(hor_res.split(' - ')[0])
    except Exception:
        error = True

    try:
        pontos_visitante = hor_res.split(' - ')[1]
        if pontos_visitante.endswith('*'):
            pontos_visitante = pontos_visitante.split('*')[0]
        
        pontos_visitante = int(pontos_visitante)
    except Exception:
        error = True

    if error:
        row['Pontos Mandante'] = 'X'
        row['Pontos Visitante'] = 'X'
        return row

    row['Pontos Mandante'] = pontos_mandante
    row['Pontos Visitante'] = pontos_visitante

    return row

def get_match_winner(row):
    def jogo_anulado(pontos_mandante, pontos_visitante):
        return pontos_mandante == 'X' and pontos_visitante == 'X'
    
    def empate(pontos_mandante, pontos_visitante):
        return pontos_mandante == pontos_visitante
    
    def vitoria_mandante(pontos_mandante, pontos_visitante):
        return pontos_mandante > pontos_visitante
    
    def vitoria_visitante(pontos_mandante, pontos_visitante):
        return pontos_mandante < pontos_visitante
    

    pontos_mandante = row['Pontos Mandante']
    pontos_visitante = row['Pontos Visitante']

    if jogo_anulado(pontos_mandante, pontos_visitante):
        vencedor = 'X'
    elif empate(pontos_mandante, pontos_visitante):
        vencedor = 'Empate'
    elif vitoria_mandante(pontos_mandante, pontos_visitante):
        vencedor = row['Mandante']
    elif vitoria_visitante(pontos_mandante, pontos_visitante):
        vencedor = row['Visitante']

    return vencedor
        

# Exploration

In [131]:
games_df = pd.read_csv('../../data/games.csv')
games_df_2 = pd.read_csv('../../data/games2.csv')
games_df_3 = pd.read_csv('../../data/games3.csv')
games_df = pd.concat([games_df, games_df_2])
games_df = pd.concat([games_df, games_df_3])
games_df.head()

,Data,Mandante,Hor/Res,Visitante,Campo,Fase,Torneio,0,Temporada
0,2024-06-29 17:00:00,Sinop Coyotes,18 - 13,Cuiabá Arsenal,Estádio Gigante do Norte - Sinop/MT,NaN,bfa-2024,NaN,NaN
1,2024-07-06 16:00:00,Rondonópolis Hawks,53 - 06,Porto Velho Miners,Rondonópolis/MT,NaN,bfa-2024,NaN,NaN
2,2024-07-07 14:00:00,Sergipe Redentores,15 - 09,Mossoró Petroleiros,Aracaju/SE,NaN,bfa-2024,NaN,NaN
3,2024-07-07 14:00:00,Recife Mariners,25 - 00,Cavalaria 2 de Julho,Estádio do Arruda - Recife/PE,NaN,bfa-2024,NaN,NaN
4,2024-07-07 14:00:00,Tubarões do Cerrado,10 - 28,Manaus FA,Estádio Rorizão - Brasília/DF,NaN,bfa-2024,NaN,NaN


In [132]:
len(games_df)

4682

#### Isso aqui foi provavelmente alguma coisa mal formatada. Não achei muita forma de tirar do script de scrapping, então a hora de tirar é no preprocess mesmo

In [133]:
games_df['0'].value_counts()

0
Sorriso Hornets    2
Name: count, dtype: int64

In [134]:
games_df['0'].value_counts().index.to_list()

['Sorriso Hornets']

In [135]:
games_df[games_df['0'] == 'Sorriso Hornets']

,Data,Mandante,Hor/Res,Visitante,Campo,Fase,Torneio,0,Temporada
452,NaN,NaN,NaN,NaN,NaN,NaN,bfa-acesso-2019,Sorriso Hornets,NaN
1149,NaN,NaN,NaN,NaN,NaN,NaN,liga-nacional-2018,Sorriso Hornets,NaN


In [136]:
games_df = games_df[games_df['0'] != 'Sorriso Hornets']
games_df.drop(columns=['0'], inplace=True)

In [137]:
games_df.head()

,Data,Mandante,Hor/Res,Visitante,Campo,Fase,Torneio,Temporada
0,2024-06-29 17:00:00,Sinop Coyotes,18 - 13,Cuiabá Arsenal,Estádio Gigante do Norte - Sinop/MT,NaN,bfa-2024,NaN
1,2024-07-06 16:00:00,Rondonópolis Hawks,53 - 06,Porto Velho Miners,Rondonópolis/MT,NaN,bfa-2024,NaN
2,2024-07-07 14:00:00,Sergipe Redentores,15 - 09,Mossoró Petroleiros,Aracaju/SE,NaN,bfa-2024,NaN
3,2024-07-07 14:00:00,Recife Mariners,25 - 00,Cavalaria 2 de Julho,Estádio do Arruda - Recife/PE,NaN,bfa-2024,NaN
4,2024-07-07 14:00:00,Tubarões do Cerrado,10 - 28,Manaus FA,Estádio Rorizão - Brasília/DF,NaN,bfa-2024,NaN


In [138]:
len(games_df)

4680

#### Por algum motivo, na tabela de jogos da Conferência Norte do Campeonato Brasileiro de 2013 tinha de fato essa coluna Temporada. Como não era um card, ele não filtrava as informações que queria, só jogava a tabela do jeito que tava. Só remover essa coluna aqui

In [139]:
games_df['Temporada'].value_counts()

Temporada
2013.0    2
Name: count, dtype: int64

In [140]:
games_df[games_df['Temporada'] == 2013.0]

,Data,Mandante,Hor/Res,Visitante,Campo,Fase,Torneio,Temporada
994,2013-08-18 14:00:00,Manaus Cavaliers,48 - 02,Belém Titans,Manaus/AM,NaN,campeonato-brasileiro-2013,2013.0
995,2013-09-15 14:00:00,Belém Titans,08 - 50,Manaus Cavaliers,Belém/PA,NaN,campeonato-brasileiro-2013,2013.0


In [141]:
games_df = games_df[games_df['Temporada'] != 2013.0]
games_df.drop(columns=['Temporada'], inplace=True)

In [142]:
games_df.head()

,Data,Mandante,Hor/Res,Visitante,Campo,Fase,Torneio
0,2024-06-29 17:00:00,Sinop Coyotes,18 - 13,Cuiabá Arsenal,Estádio Gigante do Norte - Sinop/MT,NaN,bfa-2024
1,2024-07-06 16:00:00,Rondonópolis Hawks,53 - 06,Porto Velho Miners,Rondonópolis/MT,NaN,bfa-2024
2,2024-07-07 14:00:00,Sergipe Redentores,15 - 09,Mossoró Petroleiros,Aracaju/SE,NaN,bfa-2024
3,2024-07-07 14:00:00,Recife Mariners,25 - 00,Cavalaria 2 de Julho,Estádio do Arruda - Recife/PE,NaN,bfa-2024
4,2024-07-07 14:00:00,Tubarões do Cerrado,10 - 28,Manaus FA,Estádio Rorizão - Brasília/DF,NaN,bfa-2024


In [143]:
len(games_df)

4678

#### O assert passou, então todos os jogos estão com datetime correto

In [144]:
assert len(games_df) == games_df['Data'].apply(lambda x: is_datetime(x)).sum()

#### Checando as URLs

In [145]:
urls = ['http://www.salaooval.com.br/campeonatos/bfa-2024/', 'http://www.salaooval.com.br/campeonatos/bfa-2023/', 'http://www.salaooval.com.br/campeonatos/bfa-2022/', 'http://www.salaooval.com.br/campeonatos/bfa-2019/', 'http://www.salaooval.com.br/campeonatos/bfa-2018/', 'http://www.salaooval.com.br/campeonatos/bfa-2017/', 'http://www.salaooval.com.br/campeonatos/bfa-acesso-2019/', 'https://www.salaooval.com.br/campeonatos/brasileirao-2024/', 'https://www.salaooval.com.br/campeonatos/brasileirao-2023/', 'https://www.salaooval.com.br/campeonatos/brasileirao-2022/', 'http://www.salaooval.com.br/campeonatos/superliga-nacional-2016/', 'http://www.salaooval.com.br/campeonatos/superliga-nacional-2015/', 'http://www.salaooval.com.br/campeonatos/superliga-nacional-2014/', 'http://www.salaooval.com.br/campeonatos/campeonato-brasileiro-2013/', 'http://www.salaooval.com.br/campeonatos/campeonato-brasileiro-2012/', 'http://www.salaooval.com.br/campeonatos/liga-brasileira-2011/', 'http://www.salaooval.com.br/campeonatos/liga-brasileira-2010/', 'http://www.salaooval.com.br/campeonatos/liga-nacional-2018/', 'http://www.salaooval.com.br/campeonatos/liga-nacional-2017/', 'http://www.salaooval.com.br/campeonatos/liga-nacional-2016/', 'http://www.salaooval.com.br/campeonatos/liga-nacional-2015/', 'http://www.salaooval.com.br/campeonatos/liga-nacional-2014/', 'http://www.salaooval.com.br/campeonatos/torneio-touchdown-2015/', 'http://www.salaooval.com.br/campeonatos/torneio-touchdown-2014/', 'http://www.salaooval.com.br/campeonatos/torneio-touchdown-2013/', 'http://www.salaooval.com.br/campeonatos/torneio-touchdown-2012/', 'http://www.salaooval.com.br/campeonatos/torneio-touchdown-2011/', 'http://www.salaooval.com.br/campeonatos/torneio-touchdown-2010/', 'http://www.salaooval.com.br/campeonatos/torneio-touchdown-2009/', 'http://www.salaooval.com.br/campeonatos/campeonato-amazonense-2019/', 'http://www.salaooval.com.br/campeonatos/campeonato-amazonense-2017/', 'http://www.salaooval.com.br/campeonatos/campeonato-amazonense-2016/', 'http://www.salaooval.com.br/campeonatos/campeonato-amazonense-2015/', 'http://www.salaooval.com.br/campeonatos/campeonato-amazonense-2014/', 'http://www.salaooval.com.br/campeonatos/campeonato-paraense-2019/', 'http://www.salaooval.com.br/campeonatos/campeonato-paraense-2018/', 'http://www.salaooval.com.br/campeonatos/campeonato-paraense-2017/', 'http://www.salaooval.com.br/campeonatos/campeonato-paraense-2016/', 'http://www.salaooval.com.br/campeonatos/campeonato-pernambucano-2017/', 'http://www.salaooval.com.br/campeonatos/campeonato-candango-2019/', 'http://www.salaooval.com.br/campeonatos/campeonato-candango-2016/', 'http://www.salaooval.com.br/campeonatos/campeonato-candango-2015/', 'http://www.salaooval.com.br/campeonatos/campeonato-candango-2014/', 'http://www.salaooval.com.br/campeonatos/campeonato-mato-grossense-2018/', 'http://www.salaooval.com.br/campeonatos/campeonato-matogrossense-2017/', 'http://www.salaooval.com.br/campeonatos/campeonato-matogrossense-2016/', 'http://www.salaooval.com.br/campeonatos/campeonato-sul-matogrossense-2018/', 'http://www.salaooval.com.br/campeonatos/sao-paulo-football-league-2024/', 'http://www.salaooval.com.br/campeonatos/sao-paulo-football-league-2023/', 'http://www.salaooval.com.br/campeonatos/sao-paulo-football-league-2022/', 'http://www.salaooval.com.br/campeonatos/sao-paulo-football-league-2019/', 'http://www.salaooval.com.br/campeonatos/sao-paulo-football-league-2018/', 'http://www.salaooval.com.br/campeonatos/sao-paulo-football-league-2017/', 'http://www.salaooval.com.br/campeonatos/sao-paulo-football-league/', 'http://www.salaooval.com.br/campeonatos/super-copa-sao-paulo-2017/', 'http://www.salaooval.com.br/campeonatos/super-copa-sao-paulo-2016/', 'http://www.salaooval.com.br/campeonatos/super-copa-sao-paulo-2015/', 'http://www.salaooval.com.br/campeonatos/super-copa-sao-paulo-2014/', 'http://www.salaooval.com.br/campeonatos/super-copa-sao-paulo-2013/', 'http://www.salaooval.com.br/campeonatos/super-copa-sao-paulo-2012/', 'http://www.salaooval.com.br/campeonatos/campeonato-paulista-2014/', 'http://www.salaooval.com.br/campeonatos/campeonato-paulista-2013/', 'http://www.salaooval.com.br/campeonatos/campeonato-paulista-2012/', 'http://www.salaooval.com.br/campeonatos/campeonato-paulista-2011/', 'http://www.salaooval.com.br/campeonatos/campeonato-paulista-2010/', 'http://www.salaooval.com.br/campeonatos/campeonato-mineiro-2024/', 'http://www.salaooval.com.br/campeonatos/campeonato-mineiro-2023/', 'http://www.salaooval.com.br/campeonatos/campeonato-mineiro-2022/', 'http://www.salaooval.com.br/campeonatos/campeonato-mineiro-2019/', 'http://www.salaooval.com.br/campeonatos/campeonato-mineiro-2018/', 'http://www.salaooval.com.br/campeonatos/campeonato-mineiro-2016/', 'http://www.salaooval.com.br/campeonatos/campeonato-mineiro-2012/', 'http://www.salaooval.com.br/campeonatos/campeonato-carioca-2019/', 'http://www.salaooval.com.br/campeonatos/campeonato-carioca-2018/', 'http://www.salaooval.com.br/campeonatos/liga-fluminense-2017/', 'http://www.salaooval.com.br/campeonatos/liga-fluminense-2016/', 'http://www.salaooval.com.br/campeonatos/campeonato-capixaba-2015/', 'http://www.salaooval.com.br/campeonatos/campeonato-capixaba-2013/', 'http://www.salaooval.com.br/campeonatos/campeonato-capixaba-2012/', 'http://www.salaooval.com.br/campeonatos/campeonato-paranaense-2024/', 'http://www.salaooval.com.br/campeonatos/campeonato-paranaense-2023/', 'http://www.salaooval.com.br/campeonatos/campeonato-paranaense-2022/', 'http://www.salaooval.com.br/campeonatos/campeonato-paranaense-2019/', 'http://www.salaooval.com.br/campeonatos/campeonato-paranaense-2018/', 'http://www.salaooval.com.br/campeonatos/campeonato-paranaense-2017/', 'http://www.salaooval.com.br/campeonatos/campeonato-paranaense-2016/', 'http://www.salaooval.com.br/campeonatos/campeonato-paranaense-2015/', 'http://www.salaooval.com.br/campeonatos/campeonato-paranaense-2014/', 'http://www.salaooval.com.br/campeonatos/campeonato-paranaense-2013/', 'http://www.salaooval.com.br/campeonatos/campeonato-paranaense-2012/', 'http://www.salaooval.com.br/campeonatos/campeonato-paranaense-2011/', 'http://www.salaooval.com.br/campeonatos/campeonato-paranaense-2010/', 'http://www.salaooval.com.br/campeonatos/campeonato-paranaense-2009/', 'http://www.salaooval.com.br/campeonatos/campeonato-catarinense-2019/', 'http://www.salaooval.com.br/campeonatos/campeonato-catarinense-2018/', 'http://www.salaooval.com.br/campeonatos/campeonato-catarinense-2017/', 'http://www.salaooval.com.br/campeonatos/campeonato-catarinense-2016/', 'http://www.salaooval.com.br/campeonatos/campeonato-catarinense-2015/', 'http://www.salaooval.com.br/campeonatos/campeonato-catarinense-2014/', 'http://www.salaooval.com.br/campeonatos/campeonato-catarinense-2013/', 'http://www.salaooval.com.br/campeonatos/campeonato-catarinense-2012/', 'http://www.salaooval.com.br/campeonatos/campeonato-catarinense-2011/', 'http://www.salaooval.com.br/campeonatos/campeonato-catarinense-2010/', 'http://www.salaooval.com.br/campeonatos/campeonato-catarinense-2009/', 'http://www.salaooval.com.br/campeonatos/campeonato-gaucho-2023/', 'http://www.salaooval.com.br/campeonatos/campeonato-gaucho-2022/', 'http://www.salaooval.com.br/campeonatos/campeonato-gaucho-2019/', 'http://www.salaooval.com.br/campeonatos/campeonato-gaucho-2018/', 'http://www.salaooval.com.br/campeonatos/campeonato-gaucho-2017/', 'http://www.salaooval.com.br/campeonatos/campeonato-gaucho-2016/', 'http://www.salaooval.com.br/campeonatos/campeonato-gaucho-2015/', 'http://www.salaooval.com.br/campeonatos/campeonato-gaucho-2014/', 'http://www.salaooval.com.br/campeonatos/campeonato-gaucho-2013/', 'http://www.salaooval.com.br/campeonatos/campeonato-gaucho-2012/', 'http://www.salaooval.com.br/campeonatos/bfa-norte-2019/', 'http://www.salaooval.com.br/campeonatos/copa-central-2017/', 'http://www.salaooval.com.br/campeonatos/copa-nordeste-2023/', 'http://www.salaooval.com.br/campeonatos/copa-nordeste-2022/', 'http://www.salaooval.com.br/campeonatos/copa-nordeste-2019/', 'http://www.salaooval.com.br/campeonatos/copa-nordeste-2018/', 'http://www.salaooval.com.br/campeonatos/torneio-velho-chico-2019/', 'http://www.salaooval.com.br/campeonatos/torneio-velho-chico-2018/', 'http://www.salaooval.com.br/campeonatos/torneio-velho-chico-2017/', 'http://www.salaooval.com.br/campeonatos/liga-nordeste-2018/', 'http://www.salaooval.com.br/campeonatos/liga-nordeste-2017/', 'http://www.salaooval.com.br/campeonatos/liga-nordeste-2016/', 'http://www.salaooval.com.br/campeonatos/liga-nordeste-2011/', 'http://www.salaooval.com.br/campeonatos/copa-norte-2017/', 'http://www.salaooval.com.br/campeonatos/taca-das-cidades-2018/', 'http://www.salaooval.com.br/campeonatos/taca-pernambuco-2016/', 'http://www.salaooval.com.br/campeonatos/taca-nove-de-julho-2022/', 'http://www.salaooval.com.br/campeonatos/taca-nove-de-julho-2018/', 'http://www.salaooval.com.br/campeonatos/taca-nove-de-julho-2017/', 'http://www.salaooval.com.br/campeonatos/taca-nove-de-julho-2016/', 'http://www.salaooval.com.br/campeonatos/taca-nove-de-julho-2015/', 'http://www.salaooval.com.br/campeonatos/taca-nove-de-julho-2014/', 'http://www.salaooval.com.br/campeonatos/taca-nove-de-julho-2013/', 'http://www.salaooval.com.br/campeonatos/pick-six-cup-2023/', 'http://www.salaooval.com.br/campeonatos/pick-six-cup-2022/', 'http://www.salaooval.com.br/campeonatos/pick-six-cup-2019/', 'http://www.salaooval.com.br/campeonatos/pick-six-cup-2018/', 'http://www.salaooval.com.br/campeonatos/pick-six-cup-2017/', 'https://www.salaooval.com.br/campeonatos/spring-cup-2022/', 'http://www.salaooval.com.br/campeonatos/copa-mogiana-2019/', 'http://www.salaooval.com.br/campeonatos/copa-mogiana-2018/', 'http://www.salaooval.com.br/campeonatos/copa-mogiana-2017/', 'http://www.salaooval.com.br/campeonatos/copa-ouro-2019/', 'http://www.salaooval.com.br/campeonatos/copa-ouro-2018/', 'http://www.salaooval.com.br/campeonatos/copa-ouro-2017/', 'http://www.salaooval.com.br/campeonatos/taca-rio-2019/', 'http://www.salaooval.com.br/campeonatos/taca-rio-2018/', 'https://www.salaooval.com.br/campeonatos/ctt-summer-festival-2021/', 'http://www.salaooval.com.br/campeonatos/freedom-four-cup-2016/', 'http://www.salaooval.com.br/campeonatos/freedom-four-cup-2015/', 'http://www.salaooval.com.br/campeonatos/taca-sao-paulo-2011/', 'http://www.salaooval.com.br/campeonatos/copa-minas-2017/', 'http://www.salaooval.com.br/campeonatos/copa-anhanguera-2014/', 'https://www.salaooval.com.br/campeonatos/copa-pr-2023/', 'https://www.salaooval.com.br/campeonatos/copa-pr-2022/', 'http://www.salaooval.com.br/campeonatos/copa-sul-2019/', 'http://www.salaooval.com.br/campeonatos/copa-sul-2018/', 'http://www.salaooval.com.br/campeonatos/copa-sul-2017/', 'http://www.salaooval.com.br/campeonatos/copa-sul-2016/', 'http://www.salaooval.com.br/campeonatos/copa-sul-2015/', 'http://www.salaooval.com.br/campeonatos/copa-sul-2014/', 'http://www.salaooval.com.br/campeonatos/copa-sul-2013/', 'http://www.salaooval.com.br/campeonatos/copa-fronteira-2019/', 'http://www.salaooval.com.br/campeonatos/copa-fronteira-2018/', 'http://www.salaooval.com.br/campeonatos/copa-fronteira-2017/', 'http://www.salaooval.com.br/campeonatos/copa-fronteira-2016/', 'http://www.salaooval.com.br/campeonatos/copa-rs-2019/', 'http://www.salaooval.com.br/campeonatos/copa-rs-2018/', 'http://www.salaooval.com.br/campeonatos/copa-rs-2017/', 'http://www.salaooval.com.br/campeonatos/copa-rs-2016/', 'https://www.salaooval.com.br/campeonatos/taca-nove-de-julho-2023/', 'https://www.salaooval.com.br/campeonatos/campeonato-mato-grossense-2015/', 'http://www.salaooval.com.br/campeonatos/campeonato-mato-grossense-2019/']

In [146]:
len(urls)

177

Aparentemente algumas URLs estão faltando

In [147]:
# assert len(games_df['Torneio'].value_counts()) == len(urls)

In [148]:
campeonatos_split_normal = [url for url in urls if url.endswith('/')]

campeonatos_split_normal = [url.split('/')[-2] for url in campeonatos_split_normal]

print(campeonatos_split_normal[:5])

campeonatos = campeonatos_split_normal

['bfa-2024', 'bfa-2023', 'bfa-2022', 'bfa-2019', 'bfa-2018']


In [149]:
campeonatos_com_jogos = games_df['Torneio'].value_counts().index.to_list()
campeonatos_com_jogos[:5]

['bfa-2019',
 'bfa-2018',
 'campeonato-brasileiro-2012',
 'superliga-nacional-2016',
 'liga-nacional-2018']

In [150]:
len(campeonatos_com_jogos)

177

In [151]:
campeonatos_sem_jogos = [campeonato for campeonato in campeonatos if campeonato not in campeonatos_com_jogos]
len(campeonatos_sem_jogos)

2

Aparentemente tem 2 campeonatos que ficaram sem jogos

Vou dar uma explorada em alguns jogos pra ver se eles estão na tabela ou se de fato não foi anotado


De fato aqui não tem o que fazer não

Taça Nove de Julho 2022 - Erro 404 - Não achei registros também

Copa Fronteira 2018 - Erro 404 - Não achei nenhum registro dessa competição


Ou seja, tudo certo. Erro do Salão Oval ai aparentemente

In [152]:
urls_sem_jogos = [url for url in urls if url.split('/')[-1] in campeonatos_sem_jogos or url.split('/')[-2] in campeonatos_sem_jogos]
print(len(urls_sem_jogos))
print(urls_sem_jogos)

2
['http://www.salaooval.com.br/campeonatos/taca-nove-de-julho-2022/', 'http://www.salaooval.com.br/campeonatos/copa-fronteira-2018/']


#### Tirando instâncias duplicadas

In [153]:
games_df = games_df.drop_duplicates(subset=['Data', 'Mandante', 'Hor/Res', 'Visitante', 'Torneio'])

In [154]:
len(games_df)

4567

#### Separar resultado em 2 colunas

In [155]:
games_df.head()

,Data,Mandante,Hor/Res,Visitante,Campo,Fase,Torneio
0,2024-06-29 17:00:00,Sinop Coyotes,18 - 13,Cuiabá Arsenal,Estádio Gigante do Norte - Sinop/MT,NaN,bfa-2024
1,2024-07-06 16:00:00,Rondonópolis Hawks,53 - 06,Porto Velho Miners,Rondonópolis/MT,NaN,bfa-2024
2,2024-07-07 14:00:00,Sergipe Redentores,15 - 09,Mossoró Petroleiros,Aracaju/SE,NaN,bfa-2024
3,2024-07-07 14:00:00,Recife Mariners,25 - 00,Cavalaria 2 de Julho,Estádio do Arruda - Recife/PE,NaN,bfa-2024
4,2024-07-07 14:00:00,Tubarões do Cerrado,10 - 28,Manaus FA,Estádio Rorizão - Brasília/DF,NaN,bfa-2024


In [156]:
games_df['Hor/Res'].value_counts()

Hor/Res
49 - 00     80
00 - 49     48
21 - 00     32
42 - 00     28
20 - 00     28
            ..
06 - 05      1
44 - 26*     1
03 - 46      1
26 - 29      1
02 - 09      1
Name: count, Length: 1384, dtype: int64

In [157]:
games_df['Hor/Res'].value_counts().iloc[500:600].head(50)

Hor/Res
12 - 19    3
41 - 14    3
16 - 17    2
40 - 13    2
36 - 02    2
07 - 55    2
28 - 15    2
0 - 0      2
30 - 27    2
27 - 17    2
02 - 00    2
37 - 15    2
32 - 03    2
40 - 14    2
45 - 03    2
15 - 38    2
66 - 00    2
00 - 74    2
53 - 08    2
15 - 33    2
18 - 07    2
22 - 16    2
14 - 11    2
13 - 27    2
19 - 26    2
28 - 02    2
28 - 03    2
18 - 35    2
23 - 03    2
06 - 29    2
02 - 37    2
14 - 14    2
12 - 03    2
50 - 07    2
26 - 22    2
27 - 08    2
07 - 07    2
37 - 12    2
08 - 25    2
08 - 16    2
41 - 26    2
13 - 43    2
13 - 29    2
14 - 39    2
07 - 54    2
00 - 78    2
25 - 13    2
12 - 08    2
12 - 28    2
02 - 34    2
Name: count, dtype: int64

Esse jogo aqui, fui checar e ele terminou devido a condições desfavoráveis. O juiz terminou o jogo. Ou seja, existem situações em que isso pode ocorrer, e devem estar marcadas com estrela no final. Acho revelante salvar informação no Hor/Res, mas pode ser omitida no split do resultado entre Mandante e Visitante

In [158]:
games_df[games_df['Hor/Res'] == '44 - 26*']

,Data,Mandante,Hor/Res,Visitante,Campo,Fase,Torneio
852,2017-04-23 14:30:47,Bento Gonçalves Snakes,44 - 26*,Porto Alegre Pumpkins,Estádio da Montanha - Bento Gonçalves/RS,NaN,campeonato-gaucho-2017


No caso só esse jogo foi marcado assim

In [159]:
games_df[games_df['Hor/Res'].str.contains(r'\*')]

,Data,Mandante,Hor/Res,Visitante,Campo,Fase,Torneio
852,2017-04-23 14:30:47,Bento Gonçalves Snakes,44 - 26*,Porto Alegre Pumpkins,Estádio da Montanha - Bento Gonçalves/RS,NaN,campeonato-gaucho-2017


Verei agora quais jogos não conseguem fazer direito o split de Pontos Mandante e Pontos Visitante

Evidentemente tem alguns que foram cancelados, outros adiados e tem o caso mecnionado acima

Investigar os X - X (que se não me engano também é adiamento) e os 2 casos que ele tá só com o horário

In [160]:
games_df = games_df.apply(split_mandante_visitante, axis=1)

Os jogos ou foram adiados ou foram cancelados

In [161]:
games_df[games_df['Hor/Res'] == 'X - X']

,Data,Mandante,Hor/Res,Visitante,Campo,Fase,Torneio,Pontos Mandante,Pontos Visitante
1854,2019-06-30 00:00:00,Manaus FA,X - X,North Lions,NaN,NaN,campeonato-amazonense-2019,X,X
2082,2018-07-29 18:00:00,Campo Grande Predadores,X - X,Campo Grande Gravediggers,NaN,NaN,campeonato-sul-matogrossense-2018,X,X
3029,2018-04-29 16:00:11,Arcoverde Templários,X - X,Caruaru Wolves,NaN,NaN,taca-das-cidades-2018,X,X


São 2 partidas bem aleatórias ai que só tem o horário mesmo no lugar do placar. A própria contagem de pontos da tabela de classificação mostra que os times envolvidos tem de fato uma partida a penos. Além disso, pesquisei algum registro das partidas e não encontrei. Então suponho que elas não rolaram mesmo

In [162]:
games_df[games_df['Hor/Res'] == '14:00:0014:00']

,Data,Mandante,Hor/Res,Visitante,Campo,Fase,Torneio,Pontos Mandante,Pontos Visitante
1938,2014-08-31 14:00:00,North Lions,14:00:0014:00,Amazon Black Hawks,NaN,NaN,campeonato-amazonense-2014,X,X
3534,2013-10-05 14:00:00,Ponta Grossa Phantoms,14:00:0014:00,Itapema White Sharks,Palmeira/PR,NaN,copa-sul-2013,X,X


In [163]:
games_df[games_df['Hor/Res'] == '44 - 26*']

,Data,Mandante,Hor/Res,Visitante,Campo,Fase,Torneio,Pontos Mandante,Pontos Visitante
852,2017-04-23 14:30:47,Bento Gonçalves Snakes,44 - 26*,Porto Alegre Pumpkins,Estádio da Montanha - Bento Gonçalves/RS,NaN,campeonato-gaucho-2017,44,26


In [164]:
games_df = games_df[games_df['Pontos Mandante'] != 'X']

In [165]:
len(games_df)

4549

#### Indicar vencedor

In [166]:
games_df['Vencedor'] = games_df.apply(get_match_winner, axis=1)

In [167]:
games_df.head()

,Data,Mandante,Hor/Res,Visitante,Campo,Fase,Torneio,Pontos Mandante,Pontos Visitante,Vencedor
0,2024-06-29 17:00:00,Sinop Coyotes,18 - 13,Cuiabá Arsenal,Estádio Gigante do Norte - Sinop/MT,NaN,bfa-2024,18,13,Sinop Coyotes
1,2024-07-06 16:00:00,Rondonópolis Hawks,53 - 06,Porto Velho Miners,Rondonópolis/MT,NaN,bfa-2024,53,6,Rondonópolis Hawks
2,2024-07-07 14:00:00,Sergipe Redentores,15 - 09,Mossoró Petroleiros,Aracaju/SE,NaN,bfa-2024,15,9,Sergipe Redentores
3,2024-07-07 14:00:00,Recife Mariners,25 - 00,Cavalaria 2 de Julho,Estádio do Arruda - Recife/PE,NaN,bfa-2024,25,0,Recife Mariners
4,2024-07-07 14:00:00,Tubarões do Cerrado,10 - 28,Manaus FA,Estádio Rorizão - Brasília/DF,NaN,bfa-2024,10,28,Manaus FA


#### Ordenar as partidas pela data mais antiga

In [168]:
games_df_ascended = games_df.sort_values(by=['Data'], ascending=True)

In [169]:
games_df_ascended.head()

,Data,Mandante,Hor/Res,Visitante,Campo,Fase,Torneio,Pontos Mandante,Pontos Visitante,Vencedor
1834,2009-08-07 14:00:00,JEC Gladiators,08 - 21,Brown Spiders,NaN,NaN,torneio-touchdown-2009,8,21,Brown Spiders
740,2009-08-15 14:00:00,JEC Gladiators,12 - 06,Istepôs FA,NaN,NaN,campeonato-catarinense-2009,12,6,JEC Gladiators
1840,2009-08-15 14:00:00,São Paulo Storm,16 - 12,Sorocaba Vipers,NaN,NaN,torneio-touchdown-2009,16,12,São Paulo Storm
741,2009-08-16 14:00:00,Brusque Admirals,08 - 14,Timbó Rex,NaN,NaN,campeonato-catarinense-2009,8,14,Timbó Rex
1835,2009-08-22 14:00:00,Brown Spiders,20 - 23,Coritiba Crocodiles,NaN,NaN,torneio-touchdown-2009,20,23,Coritiba Crocodiles


In [170]:
len(games_df_ascended)

4549

# Passos para Pré Processamento

Ler todas as tabelas de jogos:
    Iterar sobre a pasta de data e achar todos os arquivos games*.csv

Remover as instâncias em que a coluna 0 é Sorriso Hornets e depois removê-la

Remover as instâncias em que a temporada é igual a 2013.0 e depois removê-la

Sanity Check de se todos os datetimes estão corretos (importantíssimo para ordenação da tabela)

Split de Hor/Res em colunas Pontos Mandante e Pontos Visitante
    - Excluir partidas em que não dá pra fazer o split
    - Considerar caso de Hor/Res com resultado correto, porém * no final

Criar coluna que indica quem venceu a partida

Ordenação de partidas pela data mais antiga
